In [ ]:
import sys;
import json;

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import Session
from snowflake.snowpark.functions import hash, sha2, lit, concat_ws, col, current_timestamp, array_construct
from snowflake.snowpark.types import *

session = get_active_session()

# Get JSON input parameters
# the result is a Python dictionary:
y = json.loads(sys.argv[0])


src_schema ='SILVER_LAYER'# Absolute name (database.schema)
target_schema = 'GOLD_LAYER'
src_table = y["silver_table"]
target_table = y["gold_table"]
functionalKey_name = y["silver_functionalKey_name"]

print("src_schema param: " + src_schema)
print("src_table param: " + src_table)
print("target_schema param: " + target_schema)
print("target_table param: " + target_table)
print("functionalKey_name param: " + functionalKey_name)

In [ ]:
select * from {{ src_table }};

In [ ]:
res_table_df = get_data_from_silver_table.to_df().drop(functionalKey_name).with_column("SYS_DATE_UPDATE",current_timestamp())

res_table_df.write.mode("truncate").save_as_table(target_table)

In [ ]:
SELECT EXECUTION_STATUS as status,ERROR_MESSAGE  as first_error,  ROWS_PRODUCED as rows_parsed, ROWS_INSERTED as rows_loaded
FROM table(information_schema.QUERY_HISTORY_BY_SESSION())
WHERE QUERY_ID = LAST_QUERY_ID();

In [ ]:
EXECUTE IMMEDIATE $$
DECLARE
  status_number NUMBER := (SELECT count(*) from {{ get_insert_results }} where status != 'SUCCESS') ;
  LOAD_NOT_COMPLETE_EXCEPTION EXCEPTION (-20002, 'TABLE NOT LOADED CORRECTLY, CHECK MONITORING TABLE PLZ!');
BEGIN
  IF (status_number != 0) THEN
    RAISE LOAD_NOT_COMPLETE_EXCEPTION;
  END IF;
EXCEPTION
  WHEN OTHER THEN
    RAISE; -- Raise the same exception that you are handling.
END;
$$;